In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import requests
import random
import csv
from dotenv import load_dotenv   #for python-dotenv method
load_dotenv()                    #for python-dotenv method
import os

ON CHARGE LES LISTES DES LIGNES ET DES ARRETS


In [2]:
df_lignes = pd.read_csv('src/referentiel-des-lignes.csv', sep=';', index_col='ID_Line')
df_arrets = pd.read_csv('src/arrets.csv', sep=';', index_col='ArRId')

LES DONNEES QU'ON VEUT UTILISER POUR L'INSTANT

In [3]:
ar_prof_vaillant = '21611'
ar_bondy_rer = '41013'
ar_fdc = '463048'
ar_ob = '463059' #> Bobigny
ar_ob2='21905' #> Place d'Italie

# Tous les arrêts de Bondy
liste_arrets_bondy = df_arrets[df_arrets['ArRTown']=='Bondy'].reset_index()
liste_arrets_bondy_li = liste_arrets_bondy['ArRId'].to_list()

li_E = 'C01729'
li_105 = 'C01134'
li_5 = 'C01375'
li_9 ='C01379'

# ON GARDE UNE LISTE DES INFOS RECUPEREES 
infos_messages = []
infos_horaires = []

FONCTIONS DE RECHERCHE DES INFOS D'UNE LIGNE ET D'UN ARRET A PARTIR DE SON CODE

In [4]:
def get_line_name(s):
    line = s[-7:-1]
    if (pd.isnull(df_lignes.loc[line]['NetworkName'])):
         ret = df_lignes.loc[line]['TransportMode'] +' '+f"{df_lignes.loc[line]['ShortName_Line']}"
    else: 
        ret = df_lignes.loc[line]['NetworkName']+' '+f"{df_lignes.loc[line]['ShortName_Line']}"
        
    return ret

def get_stop_name(s):
    arret = s[-6:-1]
    nom = df_arrets.loc[int(arret)]['ArRName']
    return nom

"MESSAGES AFFICHES SUR LES ECRANS"

In [5]:
url = "https://prim.iledefrance-mobilites.fr/marketplace/general-message?"
headers = {'apiKey': os.environ.get('RATP')}
p = {'StopPointRef' : f'STIF:StopPoint:Q:{ar_bondy_rer}:'}
#p = {'LineRef':f'STIF:Line::{li_E}:'}

response = requests.get(url, headers=headers, params=p)

In [ ]:
response.json()

In [6]:
isMessages = len(response.json()['Siri']['ServiceDelivery']['GeneralMessageDelivery'][0]['InfoMessage'])>0
list_messages=[]
if isMessages:
    arret = response.json()['Siri']['ServiceDelivery']['GeneralMessageDelivery'][0]['InfoMessage'][0]['Content']['StopPointRef'][0]['value']
    list_messages = response.json().get("Siri").get("ServiceDelivery").get('GeneralMessageDelivery')[0].get('InfoMessage')

for i in list_messages:
    arret_name = get_stop_name(arret)
    type_message = i['InfoChannelRef']['value']
    message = i.get('Content').get('Message')[0].get('MessageText').get('value')
    sysdate = datetime.datetime.strftime(datetime.datetime.now(),"%Y-%m-%d %H:%M:%S")
 
    infos_messages.append({"arret":arret_name, "message": message, "type_message":type_message,"sysdate": sysdate})
    #print(message)

In [7]:
infos_messages

[{'arret': 'Gare de Bondy',
  'message': 'De Magenta vers Nanterre-La-Folie : suppression',
  'type_message': 'Perturbation',
  'sysdate': '2024-11-29 10:58:20'},
 {'arret': 'Gare de Bondy',
  'message': 'Pour votre sécurité, nous vous invitons à ne laisser aucun bagage sans surveillance. Veuillez nous signaler tout colis ou bagage qui vous paraitrait abandonné.Merci de votre vigilance',
  'type_message': 'Information',
  'sysdate': '2024-11-29 10:58:20'},
 {'arret': 'Gare de Bondy',
  'message': 'RER E et Ligne P. En soirée la semaine et les week-ends, des travaux ont lieu sur vos lignes avec mise en place de bus de remplacement.',
  'type_message': 'Perturbation',
  'sysdate': '2024-11-29 10:58:20'},
 {'arret': 'Gare de Bondy',
  'message': 'En raison des conditions climatiques, les sols peuvent être glissants. Merci de rester vigilants lors de vos déplacements.',
  'type_message': 'Information',
  'sysdate': '2024-11-29 10:58:20'}]

PROCHAINS PASSAGES

In [8]:

url = "https://prim.iledefrance-mobilites.fr/marketplace/stop-monitoring?"
headers = {'apiKey': os.environ.get('RATP')}
p = {'MonitoringRef' : f'STIF:StopPoint:Q:{ar_ob}:'}


response = requests.get(url, headers=headers, params=p)

In [ ]:
response.json()

ON RECUPERE LE NOM DE L'ARRET ET LES DIFFERENTES DESTINATIONS

In [9]:
arret = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'][0]['MonitoredVehicleJourney']['MonitoredCall']['StopPointName'][0]['value']
direction = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'][0]['MonitoredVehicleJourney']['DestinationName'][0]['value']


dir_names = []
directions = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit']

for d in directions:
    dir_name =d['MonitoredVehicleJourney']['DestinationName'][0]['value']
    if dir_name not in dir_names:
        dir_names.append(dir_name)

print(f"Arrêt : {arret}")
print(f"Directions : {dir_names}")

Arrêt : Oberkampf
Directions : ['Bobigny-Pablo-Picasso']


In [9]:

list_passages = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit']

for i in list_passages:
    horaire= {}

    # HEURE PROCHAIN PASSAGE
    str_date = i['MonitoredVehicleJourney']['MonitoredCall']['ExpectedDepartureTime']
    date_and_time = datetime.datetime.strptime(str_date, "%Y-%m-%dT%H:%M:%S.%fZ")+datetime.timedelta(hours=1)
    str_prochain_passage = datetime.datetime.strftime(date_and_time,"%Y-%m-%d %H:%M:%S")

    # DIRECTION 
    direction = i['MonitoredVehicleJourney']['DestinationName'][0]['value']
   
    # LIGNE
    ligne = get_line_name(i['MonitoredVehicleJourney']['LineRef']['value'])
    if len(i['MonitoredVehicleJourney']['JourneyNote'])>0:
        ligne += f" - {i['MonitoredVehicleJourney']['JourneyNote'][0]['value']}"
    
    if True:
        print(f"Direction > {direction}")
        print(ligne)
        print(f"Prochain départ > {date_and_time}")
        print("*"*50)

    # ON SAUVEGARDE LES INFOS DANS UNE LISTE
    horaire.update({'arret':arret,
                    'ligne':ligne, 
                    'direction':direction, 
                    'passage':str_prochain_passage})
    infos_horaires.append(horaire)


Direction > Bobigny-Pablo-Picasso
metro 5
Prochain départ > 2024-11-29 10:58:14.564000
**************************************************
Direction > Bobigny-Pablo-Picasso
metro 5
Prochain départ > 2024-11-29 11:02:13.572000
**************************************************
Direction > Bobigny-Pablo-Picasso
metro 5
Prochain départ > 2024-11-29 11:05:31.625000
**************************************************
Direction > Bobigny-Pablo-Picasso
metro 5
Prochain départ > 2024-11-29 11:08:28.574000
**************************************************


In [11]:
print(infos_horaires)

[{'arret': 'Oberkampf', 'ligne': 'metro 5', 'direction': 'Bobigny-Pablo-Picasso', 'passage': '2024-11-27 12:03:38'}, {'arret': 'Oberkampf', 'ligne': 'metro 5', 'direction': 'Bobigny-Pablo-Picasso', 'passage': '2024-11-27 12:07:17'}, {'arret': 'Oberkampf', 'ligne': 'metro 5', 'direction': 'Bobigny-Pablo-Picasso', 'passage': '2024-11-27 12:10:48'}, {'arret': 'Oberkampf', 'ligne': 'metro 5', 'direction': 'Bobigny-Pablo-Picasso', 'passage': '2024-11-27 12:14:11'}]


SAUVEGARDE DES INFOS EN CSV

In [12]:
if (len(infos_messages)>0):
    with open('src/backup_messages.csv', 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=infos_messages[0].keys())
        writer.writeheader()
        writer.writerows(infos_messages)

if (len(infos_horaires)>0):
    with open('src/backup_horaires.csv', 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=infos_horaires[0].keys())
        writer.writeheader()
        writer.writerows(infos_horaires)

ESSAIS METRO

In [13]:
mask = df_arrets['ArRType']=="metro"

mask = df_arrets['ArRName']=="Oberkampf"
df_arrets[mask]

,ArRVersion,ArRCreated,ArRChanged,ArRName,ArRType,ArRXEpsg2154,ArRYEpsg2154,ArRTown,ArRPostalRegion,ArRAccessibility,ArRAudibleSignals,ArRVisualSigns,ArRFareZone,ZdAId,ArRGeopoint
ArRId,,,,,,,,,,,,,,,
463059,1022249-45193,2019-01-28T00:00:00+01:00,2021-05-04T17:56:37+02:00,Oberkampf,metro,653679,6862920,Paris 11e,75111,false,true,true,1,45193,"48.86464456515162, 2.368604562580775"
21905,1022328-45193,2014-12-29T00:00:00+01:00,2021-05-04T17:56:37+02:00,Oberkampf,metro,653674,6862913,Paris 11e,75111,false,true,true,1,45193,"48.86458119805383, 2.3685262686038393"
463054,1022319-45193,2019-01-28T00:00:00+01:00,2021-05-04T17:56:37+02:00,Oberkampf,metro,653652,6862945,Paris 11e,75111,false,true,true,1,45193,"48.86487188758151, 2.368223670880745"
21925,1022311-45193,2014-12-29T00:00:00+01:00,2021-05-04T17:56:37+02:00,Oberkampf,metro,653647,6862935,Paris 11e,75111,false,true,true,1,45193,"48.86477709850769, 2.368155845394551"


In [14]:
arret = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'][0]['MonitoredVehicleJourney']['MonitoredCall']['StopPointName'][0]['value']
directions = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit']
#
print(f"Arrêt : {arret}")
for d in directions:
    print(f"Direction : {d['MonitoredVehicleJourney']['DestinationName'][0]['value']}")
    

Arrêt : Oberkampf
Direction : Bobigny-Pablo-Picasso
Direction : Bobigny-Pablo-Picasso
Direction : Bobigny-Pablo-Picasso
Direction : Bobigny-Pablo-Picasso


In [15]:

url = "https://prim.iledefrance-mobilites.fr/marketplace/general-message?"
headers = {'apiKey': os.environ.get('RATP')}
p = {'StopPointRef' : f'STIF:StopPoint:Q:{ar_ob}:'}

response = requests.get(url, headers=headers, params=p)



In [16]:
list_messages = response.json().get("Siri").get("ServiceDelivery").get('GeneralMessageDelivery')[0].get('InfoMessage')

for i in list_messages:
    print(i.get('Content').get('Message')[0].get('MessageText').get('value'))

LISTE HORAIRES POUR LISTE d'ARRETS

In [ ]:

url = "https://prim.iledefrance-mobilites.fr/marketplace/stop-monitoring?"
headers = {'apiKey': os.environ.get('RATP')}

for l in liste_arrets_bondy_li:
    p = {'MonitoringRef' : f'STIF:StopPoint:Q:{l}:'}


    response = requests.get(url, headers=headers, params=p)
    
    try:
        arret = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'][0]['MonitoredVehicleJourney']['MonitoredCall']['StopPointName'][0]['value']
        direction = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'][0]['MonitoredVehicleJourney']['DestinationName'][0]['value']


        dir_names = []
        directions = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit']

        list_passages = response.json()['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit']

        for i in list_passages:
            horaire= {}

            # HEURE PROCHAIN PASSAGE
            str_date = i['MonitoredVehicleJourney']['MonitoredCall']['ExpectedDepartureTime']
            date_and_time = datetime.datetime.strptime(str_date, "%Y-%m-%dT%H:%M:%S.%fZ")+datetime.timedelta(hours=1)
            str_prochain_passage = datetime.datetime.strftime(date_and_time,"%Y-%m-%d %H:%M:%S")

            # DIRECTION 
            direction = i['MonitoredVehicleJourney']['DestinationName'][0]['value']
        
            # LIGNE
            ligne = get_line_name(i['MonitoredVehicleJourney']['LineRef']['value'])
            if len(i['MonitoredVehicleJourney']['JourneyNote'])>0:
                ligne += f" - {i['MonitoredVehicleJourney']['JourneyNote'][0]['value']}"
            
            if True:
                print(f"Direction > {direction}")
                print(ligne)
                print(f"Prochain départ > {date_and_time}")
                print("*"*50)

            # ON SAUVEGARDE LES INFOS DANS UNE LISTE
            horaire.update({'arret':arret,
                            'ligne':ligne, 
                            'direction':direction, 
                            'passage':str_prochain_passage})
            infos_horaires.append(horaire)

    except:
        print(f"PAS D'INFOs POUR {l}")

SAUVEGARDE DES HORAIRES

In [ ]:
if (len(infos_horaires)>0):
    with open('src/backup_horaires.csv', 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=infos_horaires[0].keys())
        writer.writeheader()
        writer.writerows(infos_horaires)